In [21]:
import pandas as pd
import numpy as np
from surprise import SVD, accuracy
from surprise import Reader, Dataset

In [22]:
rating = pd.read_csv("movie_rating.csv")
rating.head()

,critic,title,rating
0,Jack,Lady,3.0
1,Jack,Snakes,4.0
2,Jack,You Me,3.5
3,Jack,Superman,5.0
4,Jack,The Night,3.0


In [23]:
rating['critic'].value_counts()
rating['title'].value_counts()

tab=pd.crosstab(rating['critic'], rating['title'])
tab

title,Just My,Lady,Snakes,Superman,The Night,You Me,freak
critic,,,,,,,
Claudia,1,0,1,1,1,1,1
Gene,1,1,1,1,1,1,0
Jack,0,1,1,1,1,1,0
Lisa,1,1,1,1,1,1,0
Mick,1,1,1,1,1,1,0
Toby,0,0,1,1,0,1,0


In [24]:
rating_g = rating.groupby(['critic', 'title'])
rating_g.sum()
tab = rating_g.sum().unstack() # 행렬구조로 변환
tab

rating                                            
title   Just My Lady Snakes Superman The Night You Me freak
critic                                                     
Claudia     3.0  NaN    3.5      4.0       4.5    2.5   4.5
Gene        1.5  3.0    3.5      5.0       3.0    3.5   NaN
Jack        NaN  3.0    4.0      5.0       3.0    3.5   NaN
Lisa        3.0  2.5    3.5      3.5       3.0    2.5   NaN
Mick        2.0  3.0    4.0      3.0       3.0    2.0   NaN
Toby        NaN  NaN    4.5      4.0       NaN    1.0   NaN

In [25]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df=rating, reader=reader)

data

In [26]:
train = data.build_full_trainset() # 훈련셋
test = train.build_testset() # 검정셋

In [27]:
help(SVD)

model = SVD(n_factors=100, n_epochs=20, random_state=123)
model.fit(train) # model 생성

Help on class SVD in module surprise.prediction_algorithms.matrix_factorization:

class SVD(surprise.prediction_algorithms.algo_base.AlgoBase)
 |  SVD(n_factors=100, n_epochs=20, biased=True, init_mean=0, init_std_dev=0.1, lr_all=0.005, reg_all=0.02, lr_bu=None, lr_bi=None, lr_pu=None, lr_qi=None, reg_bu=None, reg_bi=None, reg_pu=None, reg_qi=None, random_state=None, verbose=False)
 |  
 |  The famous *SVD* algorithm, as popularized by `Simon Funk
 |  <https://sifter.org/~simon/journal/20061211.html>`_ during the Netflix
 |  Prize. When baselines are not used, this is equivalent to Probabilistic
 |  Matrix Factorization :cite:`salakhutdinov2008a` (see :ref:`note
 |  <unbiased_note>` below).
 |  
 |  The prediction :math:`\hat{r}_{ui}` is set as:
 |  
 |  .. math::
 |      \hat{r}_{ui} = \mu + b_u + b_i + q_i^Tp_u
 |  
 |  If user :math:`u` is unknown, then the bias :math:`b_u` and the factors
 |  :math:`p_u` are assumed to be zero. The same applies for item :math:`i`
 |  with :math:`b_

In [53]:
user_id = 'Toby' # 추천대상자
item_ids = ['The Night', 'Just My', 'Lady', 'freak'] # 추천 대상 영화
actual_rating = 0 # 실제 평점

result = []
for item_id in item_ids :
    predict = model.predict(user_id, item_id, actual_rating)
    #print(predict)
    
    #pred = np.sort(predict, axis=0)
    #print(pred)
    temp = [predict.uid, predict.iid, predict.est]
    result.append(temp)
result.sort(key=lambda x:(x[0], x[1]), reverse=True)

num = 1
for r in result[:3] :
    print('Top',num,'::',r[1])
    num+=1

Top 1 :: freak
Top 2 :: The Night
Top 3 :: Lady
